In [ ]:
pfrom google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import os
from os import path

In [ ]:
cd '/content/drive/MyDrive/Grad Project/MIT/Audios_Answers'

/content/drive/MyDrive/Grad Project/MIT/Audios_Answers


In [ ]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=b346f2316551aafe4d529940f85d8dea0b3e271632dad9d867cffd15e9883e3b
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
pip install librosa

In [ ]:
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa 

In [ ]:
def log_specgram(audio, sample_rate, window_size=10, 
                 step_size=5, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    _, _, spec = signal.spectrogram(audio, fs=sample_rate,
                                    window='hann', nperseg=nperseg, noverlap=noverlap,
                                    detrend=False)
    return np.log(spec.T.astype(np.float32) + eps)

In [ ]:
sig , ssrr = librosa.load('/content/drive/MyDrive/Grad Project/MIT/Audios_Answers/P1/P1Q2.wav')
aaa = log_specgram(sig , ssrr)
print(aaa.shape)

(7816, 111)


In [ ]:
def get_paths_list(identifier = 'P'):

  interview_paths = []
  for interview in range(90):
    for question in range(5):
      path_str = identifier + str(interview+1) + '/' + identifier + str(interview+1) + 'Q' + str(question+1) + '.wav'
      
      if path.exists(path_str):
        interview_paths.append(path_str)

  return interview_paths

In [ ]:
def get_all_paths():

  return get_paths_list('P') + get_paths_list('PP')

In [ ]:
def Max_audio_duration():

  Total_duration = 0
  Max_duration = 0
  Max_path = ''
  paths = get_all_paths();

  for path_str in paths:
    sampling_rate , signal = wav.read(path_str)
    duration = len(signal) / float(sampling_rate)
    Total_duration = Total_duration + duration
    print(duration)
    print(path_str)
    
    if duration > Max_duration:
      Max_duration = duration
      Max_path = path_str
  print('--------------------------MAX-------------------------------')
  print(Total_duration / 690)
  print(Max_duration)
  print(Max_path)

In [ ]:
Max_audio_duration()

52.992
P1/P1Q1.wav
38.99733333333333
P1/P1Q2.wav
42.00533333333333
P1/P1Q3.wav
22.997333333333334
P1/P1Q4.wav
11.989333333333333
P1/P1Q5.wav
61.013333333333335
P3/P3Q1.wav
65.00266666666667
P3/P3Q2.wav
129.00266666666667
P3/P3Q3.wav
111.01866666666666
P3/P3Q4.wav
69.01333333333334
P3/P3Q5.wav
16.0
P4/P4Q1.wav
57.00266666666667
P4/P4Q2.wav
76.992
P4/P4Q3.wav
55.01866666666667
P4/P4Q4.wav
18.005333333333333
P4/P4Q5.wav
26.005333333333333
P5/P5Q1.wav
40.0
P5/P5Q2.wav
35.008
P5/P5Q3.wav
32.0
P5/P5Q4.wav
19.008
P5/P5Q5.wav
105.984
P6/P6Q1.wav
74.98666666666666
P6/P6Q2.wav
125.01333333333334
P6/P6Q3.wav
40.0
P6/P6Q4.wav
55.01866666666667
P6/P6Q5.wav
11.008
P7/P7Q1.wav
35.008
P7/P7Q2.wav
64.0
P7/P7Q3.wav
69.99466666666666
P7/P7Q4.wav
53.99466666666667
P7/P7Q5.wav
29.994666666666667
P8/P8Q1.wav
82.00533333333334
P8/P8Q2.wav
85.01333333333334
P8/P8Q3.wav
48.0
P8/P8Q4.wav
41.00266666666667
P8/P8Q5.wav
34.986666666666665
P10/P10Q1.wav
123.98933333333333
P10/P10Q2.wav
94.99733333333333
P10/P10Q3.w

In [ ]:
def get_labels():
  labels_path = '/content/drive/MyDrive/Grad Project/MIT/turker_scores_full_interview.csv'
  data_frame = pd.read_csv(labels_path)
  data_frame = data_frame[data_frame['Worker'] == 'AGGR']
  return data_frame

In [ ]:
get_labels()

,Participant,Worker,Overall,RecommendHiring,Colleague,Engaged,Excited,EyeContact,Smiled,SpeakingRate,NoFillers,Friendly,Paused,EngagingTone,StructuredAnswers,Calm,NotStressed,Focused,Authentic,NotAwkward,Total
9,p1,AGGR,5.297316,5.106224,5.333004,5.541380,5.043890,5.866119,3.576160,4.865590,3.771665,5.254784,5.800468,5.147909,4.891580,5.351075,5.350760,5.845226,5.610513,5.477534,93.131196
19,p3,AGGR,4.414892,4.433070,5.010430,5.616076,5.601586,5.426861,6.062173,5.016286,3.332458,6.500329,4.791550,5.621231,3.912199,4.476537,5.521215,5.549829,6.014960,4.923550,92.225232
29,p4,AGGR,4.494494,4.530129,4.500707,5.494494,4.261343,3.857848,4.852150,4.738000,3.111445,5.369884,5.177447,4.392736,4.688379,5.065190,5.566084,5.324136,5.658364,4.452909,85.535740
39,p5,AGGR,5.457670,5.571558,5.772488,5.903057,4.707062,6.694276,3.920479,4.927181,5.881741,5.649119,5.886326,4.695523,5.582514,6.130488,5.916373,6.322086,6.020070,5.629838,100.667850
49,p6,AGGR,5.106512,4.831482,4.860595,5.020385,4.648259,4.331805,4.194227,4.174891,3.920329,4.495375,5.027823,4.261988,4.826245,5.828977,5.824390,5.619420,5.371858,5.295977,87.640538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,pp83,AGGR,6.045748,5.806617,6.347873,6.243224,5.806617,5.847779,6.483032,4.655062,4.838943,6.549279,5.764552,5.801601,4.878478,5.492810,5.391449,6.083629,6.559118,5.840547,104.436358
1348,pp84,AGGR,5.710073,6.020304,5.437203,5.749959,5.307262,5.564999,6.533606,4.656370,5.047772,6.529525,5.912420,5.625489,4.890114,5.112062,5.750670,5.988186,6.248887,5.169949,101.254849
1358,pp85,AGGR,5.626074,5.766592,5.791370,6.375623,5.584830,6.280263,5.050913,4.263639,4.097177,5.970750,5.643748,5.239474,4.918227,4.985095,4.998952,6.554104,6.372852,4.800594,98.320278
1368,pp86,AGGR,4.853881,4.700179,4.495230,5.513933,5.137644,5.892929,4.875432,4.645732,4.126127,5.022182,4.814251,5.208898,4.240302,5.648693,5.939815,5.078459,6.008339,5.154665,91.356689


In [ ]:
from python_speech_features import logfbank , delta
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
import librosa
from librosa.feature import mfcc , chroma_stft , melspectrogram , spectral_contrast , tonnetz

#Data Preprocessing

In [ ]:
def preprocess_data_P():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[:345]: #345
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    sampling_rate , signal = wav.read(one_path)#librosa.load(one_path) #wav.read(one_path)
    # get the average of 2 channels of the audio
    # print(one_path)
    # print(signal)
    # print(signal.shape)
    # print(sampling_rate)

    signal = np.mean(signal , axis=1)
    
    # pad sequences to be all in range of 55 seconds

    #final_signal = []
    #final_signal.append(signal)
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=1350000)
    #signal = signal[0]

    # extract features
    #mfcc_features = mfcc(signal , sampling_rate , n_mfcc=27)
    #print(mfcc_features.shape)
    #chroma_features = chroma_stft(signal , sampling_rate)
    #print(chroma_features.shape)
    #mel_features = melspectrogram(signal , sampling_rate , n_mels = 128)
    #print(mel_features.shape)
    #contrast_features = spectral_contrast(signal , sampling_rate , n_bands =  6)
    #print(contrast_features.shape)
    #tonnetz_features = tonnetz(signal , sampling_rate)
    #print(tonnetz_features.shape)

    log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)
    deltas = delta(log_mels , 2)
    deltas_deltas = delta(deltas , 2)

    #frame_features = np.concatenate((mfcc_features , chroma_features , mel_features , contrast_features , tonnetz_features) , axis = 0)
    #frame_features = np.transpose(frame_features)
    #frame_features = np.mean(frame_features , axis=0)
    #print(frame_features.shape)
    # append all features together
    frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    frame_features = np.mean(frame_features , axis=0)
    # curr_noframes = frame_features.shape[0]
    # average_noframes = 2500
    # no_features = 180
    # padded_arr = np.zeros((average_noframes , no_features))

    # if average_noframes >= curr_noframes:
    #   start_idxpad = int((average_noframes - curr_noframes) / 2)
    #   padded_arr[ start_idxpad : start_idxpad + curr_noframes , :] = frame_features
    # else:
    #   random_idxs = np.random.randint(curr_noframes , size=average_noframes)
    #   padded_arr = frame_features[random_idxs , :]
    
    # frame_features = padded_arr
    # print(frame_features.shape)

    all_data.append(frame_features)
    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/Grad Project/MIT/input_P_5Qs_logfilters.npy' , all_data)
  print(all_data.shape)


In [ ]:
preprocess_data_P()

Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13
Done : 14
Done : 15
Done : 16
Done : 17
Done : 18
Done : 19
Done : 20
Done : 21
Done : 22
Done : 23
Done : 24
Done : 25
Done : 26
Done : 27
Done : 28
Done : 29
Done : 30
Done : 31
Done : 32
Done : 33
Done : 34
Done : 35
Done : 36
Done : 37
Done : 38
Done : 39
Done : 40
Done : 41
Done : 42
Done : 43
Done : 44
Done : 45
Done : 46
Done : 47
Done : 48
Done : 49
Done : 50
Done : 51
Done : 52
Done : 53
Done : 54
Done : 55
Done : 56
Done : 57
Done : 58
Done : 59
Done : 60
Done : 61
Done : 62
Done : 63
Done : 64
Done : 65
Done : 66
Done : 67
Done : 68
Done : 69
Done : 70
Done : 71
Done : 72
Done : 73
Done : 74
Done : 75
Done : 76
Done : 77
Done : 78
Done : 79
Done : 80
Done : 81
Done : 82
Done : 83
Done : 84
Done : 85
Done : 86
Done : 87
Done : 88
Done : 89
Done : 90
Done : 91
Done : 92
Done : 93
Done : 94
Done : 95
Done : 96
Done : 97
Done : 98
Done : 99
Done : 100
Done : 1

In [ ]:
def preprocess_data_PP():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[345:]: #345
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    sampling_rate , signal = wav.read(one_path)#librosa.load(one_path) #wav.read(one_path)
    # get the average of 2 channels of the audio
    # print(one_path)
    # print(signal)
    # print(signal.shape)
    # print(sampling_rate)

    signal = np.mean(signal , axis=1)
    
    # pad sequences to be all in range of 55 seconds

    #final_signal = []
    #final_signal.append(signal)
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=1350000)
    #signal = signal[0]

    # extract features
    #mfcc_features = mfcc(signal , sampling_rate , n_mfcc=27)
    #print(mfcc_features.shape)
    #chroma_features = chroma_stft(signal , sampling_rate)
    #print(chroma_features.shape)
    #mel_features = melspectrogram(signal , sampling_rate , n_mels = 128)
    #print(mel_features.shape)
    #contrast_features = spectral_contrast(signal , sampling_rate , n_bands =  6)
    #print(contrast_features.shape)
    #tonnetz_features = tonnetz(signal , sampling_rate)
    #print(tonnetz_features.shape)

    log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)       #Wlen * sampling rate = 1200  -->sampling rate 48,000 --> number of fft points to be computed in one frame --> fft fast fourier transform used to be changed into discrete data
    deltas = delta(log_mels , 2)
    deltas_deltas = delta(deltas , 2)

    #frame_features = np.concatenate((mfcc_features , chroma_features , mel_features , contrast_features , tonnetz_features) , axis = 0)
    #frame_features = np.transpose(frame_features)
    #frame_features = np.mean(frame_features , axis=0)
    #print(frame_features.shape)
    # append all features together
    frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    frame_features = np.mean(frame_features , axis=0)
    # curr_noframes = frame_features.shape[0]
    # average_noframes = 2500
    # no_features = 180
    # padded_arr = np.zeros((average_noframes , no_features))

    # if average_noframes >= curr_noframes:
    #   start_idxpad = int((average_noframes - curr_noframes) / 2)
    #   padded_arr[ start_idxpad : start_idxpad + curr_noframes , :] = frame_features
    # else:
    #   random_idxs = np.random.randint(curr_noframes , size=average_noframes)
    #   padded_arr = frame_features[random_idxs , :]
    
    # frame_features = padded_arr
    # print(frame_features.shape)

    all_data.append(frame_features)
    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/Grad Project/MIT/input_PP_5Qs_logfilters.npy' , all_data)
  print(all_data.shape)


In [ ]:
preprocess_data_PP()

Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13
Done : 14
Done : 15
Done : 16
Done : 17
Done : 18
Done : 19
Done : 20
Done : 21
Done : 22
Done : 23
Done : 24
Done : 25
Done : 26
Done : 27
Done : 28
Done : 29
Done : 30
Done : 31
Done : 32
Done : 33
Done : 34
Done : 35
Done : 36
Done : 37
Done : 38
Done : 39
Done : 40
Done : 41
Done : 42
Done : 43
Done : 44
Done : 45
Done : 46
Done : 47
Done : 48
Done : 49
Done : 50
Done : 51
Done : 52
Done : 53
Done : 54
Done : 55
Done : 56
Done : 57
Done : 58
Done : 59
Done : 60
Done : 61
Done : 62
Done : 63
Done : 64
Done : 65
Done : 66
Done : 67
Done : 68
Done : 69
Done : 70
Done : 71
Done : 72
Done : 73
Done : 74
Done : 75
Done : 76
Done : 77
Done : 78
Done : 79
Done : 80
Done : 81
Done : 82
Done : 83
Done : 84
Done : 85
Done : 86
Done : 87
Done : 88
Done : 89
Done : 90
Done : 91
Done : 92
Done : 93
Done : 94
Done : 95
Done : 96
Done : 97
Done : 98
Done : 99
Done : 100
Done : 1

In [ ]:
def preprocess_data_PP():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[345:]: #345
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    signal , sampling_rate = librosa.load(one_path) #wav.read(one_path)
    # get the average of 2 channels of the audio
    # print(one_path)
    # print(signal)
    # print(signal.shape)
    # print(sampling_rate)

    #signal = np.mean(signal , axis=1)
    
    # pad sequences to be all in range of 55 seconds

    #final_signal = []
    #final_signal.append(signal)
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=1350000)
    #signal = signal[0]

    # extract features
    mfcc_features = mfcc(signal , sampling_rate , n_mfcc=27)
    #print(mfcc_features.shape)
    chroma_features = chroma_stft(signal , sampling_rate)
    #print(chroma_features.shape)
    mel_features = melspectrogram(signal , sampling_rate , n_mels = 128)
    #print(mel_features.shape)
    contrast_features = spectral_contrast(signal , sampling_rate , n_bands =  6)
    #print(contrast_features.shape)
    tonnetz_features = tonnetz(signal , sampling_rate)
    #print(tonnetz_features.shape)

    #log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)
    #deltas = delta(log_mels , 2)
    #deltas_deltas = delta(deltas , 2)

    frame_features = np.concatenate((mfcc_features , chroma_features , mel_features , contrast_features , tonnetz_features) , axis = 0)
    frame_features = np.transpose(frame_features)
    frame_features = np.mean(frame_features , axis=0)
    print(frame_features.shape)
    # append all features together
    #frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    #frame_features = np.mean(frame_features , axis=0)
    # curr_noframes = frame_features.shape[0]
    # average_noframes = 2500
    # no_features = 180
    # padded_arr = np.zeros((average_noframes , no_features))

    # if average_noframes >= curr_noframes:
    #   start_idxpad = int((average_noframes - curr_noframes) / 2)
    #   padded_arr[ start_idxpad : start_idxpad + curr_noframes , :] = frame_features
    # else:
    #   random_idxs = np.random.randint(curr_noframes , size=average_noframes)
    #   padded_arr = frame_features[random_idxs , :]
    
    # frame_features = padded_arr
    # print(frame_features.shape)

    all_data.append(frame_features)
    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/Grad Project/MIT/input_PP_5Qs_allframes.npy' , all_data)
  print(all_data.shape)


In [ ]:
preprocess_data_PP()

(180,)
Done : 1
(180,)
Done : 2
(180,)
Done : 3
(180,)
Done : 4
(180,)
Done : 5
(180,)
Done : 6
(180,)
Done : 7
(180,)
Done : 8
(180,)
Done : 9
(180,)
Done : 10
(180,)
Done : 11
(180,)
Done : 12
(180,)
Done : 13
(180,)
Done : 14
(180,)
Done : 15
(180,)
Done : 16
(180,)
Done : 17
(180,)
Done : 18
(180,)
Done : 19
(180,)
Done : 20
(180,)
Done : 21
(180,)
Done : 22
(180,)
Done : 23
(180,)
Done : 24
(180,)
Done : 25
(180,)
Done : 26
(180,)
Done : 27
(180,)
Done : 28
(180,)
Done : 29
(180,)
Done : 30
(180,)
Done : 31
(180,)
Done : 32
(180,)
Done : 33
(180,)
Done : 34
(180,)
Done : 35
(180,)
Done : 36
(180,)
Done : 37
(180,)
Done : 38
(180,)
Done : 39
(180,)
Done : 40
(180,)
Done : 41
(180,)
Done : 42
(180,)
Done : 43
(180,)
Done : 44
(180,)
Done : 45
(180,)
Done : 46
(180,)
Done : 47
(180,)
Done : 48
(180,)
Done : 49
(180,)
Done : 50
(180,)
Done : 51
(180,)
Done : 52
(180,)
Done : 53
(180,)
Done : 54
(180,)
Done : 55
(180,)
Done : 56
(180,)
Done : 57
(180,)
Done : 58
(180,)
Done : 59
(180,)

In [ ]:
preprocess_data_PP()

(1034, 180)
(2500, 180)
Done : 1
(1207, 180)
(2500, 180)
Done : 2
(2239, 180)
(2500, 180)
Done : 3
(991, 180)
(2500, 180)
Done : 4
(474, 180)
(2500, 180)
Done : 5
(3704, 180)
(2500, 180)
Done : 6
(4953, 180)
(2500, 180)
Done : 7
(5212, 180)
(2500, 180)
Done : 8
(5340, 180)
(2500, 180)
Done : 9
(3920, 180)
(2500, 180)
Done : 10
(2025, 180)
(2500, 180)
Done : 11
(3489, 180)
(2500, 180)
Done : 12
(4393, 180)
(2500, 180)
Done : 13
(4523, 180)
(2500, 180)
Done : 14
(4866, 180)
(2500, 180)
Done : 15
(1077, 180)
(2500, 180)
Done : 16
(2111, 180)
(2500, 180)
Done : 17
(1982, 180)
(2500, 180)
Done : 18
(1982, 180)
(2500, 180)
Done : 19
(1508, 180)
(2500, 180)
Done : 20
(7064, 180)
(2500, 180)
Done : 21
(2885, 180)
(2500, 180)
Done : 22
(5772, 180)
(2500, 180)
Done : 23
(3489, 180)
(2500, 180)
Done : 24
(1895, 180)
(2500, 180)
Done : 25
(733, 180)
(2500, 180)
Done : 26
(1507, 180)
(2500, 180)
Done : 27
(2283, 180)
(2500, 180)
Done : 28
(1809, 180)
(2500, 180)
Done : 29
(732, 180)
(2500, 180)
Don

In [ ]:
preprocess_data_P()

Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13
Done : 14
Done : 15
Done : 16
Done : 17
Done : 18
Done : 19
Done : 20
Done : 21
Done : 22
Done : 23
Done : 24
Done : 25
Done : 26
Done : 27
Done : 28
Done : 29
Done : 30
Done : 31
Done : 32
Done : 33
Done : 34
Done : 35
Done : 36
Done : 37
Done : 38
Done : 39
Done : 40
Done : 41
Done : 42
Done : 43
Done : 44
Done : 45
Done : 46
Done : 47
Done : 48
Done : 49
Done : 50
Done : 51
Done : 52
Done : 53
Done : 54
Done : 55
Done : 56
Done : 57
Done : 58
Done : 59
Done : 60
Done : 61
Done : 62
Done : 63
Done : 64
Done : 65
Done : 66
Done : 67
Done : 68
Done : 69
Done : 70
Done : 71
Done : 72
Done : 73
Done : 74
Done : 75
Done : 76
Done : 77
Done : 78
Done : 79
Done : 80
Done : 81
Done : 82
Done : 83
Done : 84
Done : 85
Done : 86
Done : 87
Done : 88
Done : 89
Done : 90
Done : 91
Done : 92
Done : 93
Done : 94
Done : 95
Done : 96
Done : 97
Done : 98
Done : 99
Done : 100
Done : 1

In [ ]:
def preprocess_data_PP():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[345:]:
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    sampling_rate , signal = wav.read(one_path)
    # get the average of 2 channels of the audio
    signal = np.mean(signal , axis=1)
    # pad sequences to be all in range of 150 seconds
    #final_signal = []
    #final_signal.append(list(signal))
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=3000000)
    #signal = signal[0]

    # extract features
    log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)
    deltas = delta(log_mels , 2)
    deltas_deltas = delta(deltas , 2)

    # append all features together
    frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    frame_features = np.mean(frame_features , axis=0)
    all_data.append(frame_features)

    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/Grad Project/MIT/input_PP_5Qs_allframes.npy' , all_data)
  print(all_data.shape)

In [ ]:
preprocess_data_PP()

Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13
Done : 14
Done : 15
Done : 16
Done : 17
Done : 18
Done : 19
Done : 20
Done : 21
Done : 22
Done : 23
Done : 24
Done : 25
Done : 26
Done : 27
Done : 28
Done : 29
Done : 30
Done : 31
Done : 32
Done : 33
Done : 34
Done : 35
Done : 36
Done : 37
Done : 38
Done : 39
Done : 40
Done : 41
Done : 42
Done : 43
Done : 44
Done : 45
Done : 46
Done : 47
Done : 48
Done : 49
Done : 50
Done : 51
Done : 52
Done : 53
Done : 54
Done : 55
Done : 56
Done : 57
Done : 58
Done : 59
Done : 60
Done : 61
Done : 62
Done : 63
Done : 64
Done : 65
Done : 66
Done : 67
Done : 68
Done : 69
Done : 70
Done : 71
Done : 72
Done : 73
Done : 74
Done : 75
Done : 76
Done : 77
Done : 78
Done : 79
Done : 80
Done : 81
Done : 82
Done : 83
Done : 84
Done : 85
Done : 86
Done : 87
Done : 88
Done : 89
Done : 90
Done : 91
Done : 92
Done : 93
Done : 94
Done : 95
Done : 96
Done : 97
Done : 98
Done : 99
Done : 100
Done : 1

#Training

In [ ]:
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm

In [ ]:
def average_5Qs(file_path):

  arr_features = np.load(file_path)
  Total_interviews = 69
  Total_questions = 5
  Total_data = []
  for i in range(Total_interviews):
    Total_data.append(np.mean(arr_features[i*5:i*5+5] , axis=0))
    print(str(i))

  
  interviews_arr = np.stack(Total_data , axis=0)
  print(interviews_arr.shape)
  return interviews_arr

In [ ]:
def save_interview_features():
  
  P_interviews = average_5Qs('/content/drive/MyDrive/Grad Project/MIT/input_P_5Qs_logfilters.npy')
  PP_interviews = average_5Qs('/content/drive/MyDrive/Grad Project/MIT/input_PP_5Qs_logfilters.npy')

  np.save('/content/drive/MyDrive/Grad Project/MIT/interview_features_logfilters.npy' , np.concatenate((P_interviews , PP_interviews)))

In [ ]:
save_interview_features()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
(69, 40, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
(69, 40, 3)


In [ ]:
def set_data(trait = 'Overall'):
  
  all_interviews = np.load('/content/drive/MyDrive/Grad Project/MIT/interview_features_logfilters.npy')
  print(all_interviews.shape)
  all_interviews = all_interviews.reshape(all_interviews.shape[0] , all_interviews.shape[1] * all_interviews.shape[2])
  print(all_interviews.shape)
  labels = get_labels()[trait]


  P_data , PP_data , P_labels , PP_labels = all_interviews[0:69 , : ] , all_interviews[69: , : ] , labels[0:69] , labels[69:]
  P_data , PP_data , P_labels , PP_labels = shuffle(P_data , PP_data , P_labels , PP_labels , random_state = 0)

  del(all_interviews)
  del(labels)

  train_data , test_data = np.concatenate((P_data[0:55 , : ] , PP_data[0:55 , : ])) , np.concatenate((P_data[55: , : ] , PP_data[55: , : ]))
  train_labels, test_labels = np.concatenate((P_labels[0:55] , PP_labels[0:55])), np.concatenate((P_labels[55:] , PP_labels[55:]))

  return train_data , test_data , train_labels , test_labels

# Train without tuning

In [ ]:
def training_SVR(trait = 'Overall'):
  
  train_data , test_data , train_labels , test_labels = set_data(trait)
  #train_data = train_data.reshape(train_data.shape[0] , train_data.shape[1] * train_data.shape[2] * train_data.shape[3])
  #test_data = test_data.reshape(test_data.shape[0] , test_data.shape[1] * test_data.shape[2] * test_data.shape[3])
  scaler = MinMaxScaler()
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  test_data = scaler.transform(test_data)


  param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear' , 'rbf']}]

  model_svr = svm.SVR()
  grids = GridSearchCV(model_svr,param_grid,cv=10)

  
  grids.fit(train_data , train_labels)
  Y_predicted = grids.predict(test_data)
  correlation_res = pearsonr(Y_predicted , test_labels)
  print('--------------------------------' + trait + '--------------------------------')
  print(correlation_res)
  print(Y_predicted)

  return grids  

In [ ]:
import pickle

In [ ]:
filename = '/content/drive/MyDrive/Grad Project/MIT/Models/Friendly_Sound.sav'
# load the model
loaded_sound_model = pickle.load(open(filename, 'rb'))

In [ ]:
train_data , test_data , train_labels , test_labels = set_data("Friendly")
  #train_data = train_data.reshape(train_data.shape[0] , train_data.shape[1] * train_data.shape[2] * train_data.shape[3])
  #test_data = test_data.reshape(test_data.shape[0] , test_data.shape[1] * test_data.shape[2] * test_data.shape[3])
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)


train_predicted = loaded_sound_model.predict(train_data)
test_predicted = loaded_sound_model.predict(test_data)


print("-----------------------Main labels-----------------------------------------")
print(train_labels)

print("-----------------------predicted-----------------------------------------")
print(train_predicted)

corr1,_ = pearsonr(train_predicted , train_labels)
print(corr1)

print("-----------------------Main test labels-----------------------------------------")
print(test_labels)

print("-----------------------predicted test-----------------------------------------")
print(test_predicted)

corr2,_ = pearsonr(test_predicted , test_labels)
print(corr2)




(138, 40, 3)
(138, 120)
-----------------------Main labels-----------------------------------------
[5.6030908  4.60665832 5.39940836 5.52960646 5.35568341 5.67565879
 5.74780274 6.16352078 6.09577254 6.61158389 6.11787182 5.45432398
 4.77171493 4.51848839 4.50753132 4.01396864 5.29175366 5.44699961
 4.49537499 6.17421775 6.48303159 5.36988386 5.8989408  6.31536812
 5.64911869 5.84817494 5.44808308 4.12027395 5.16044339 4.31347714
 4.1140197  5.87249828 5.9004966  6.08019342 4.75315305 5.26243598
 5.34561879 5.46545664 5.43353996 6.56263386 5.25478356 4.80404635
 5.50456862 5.37712386 4.6311326  6.60379495 4.46230637 6.2696609
 6.07138865 3.6279524  4.36244591 5.35486235 5.09237911 6.5008171
 5.58483008 5.96578787 4.41060254 5.4590183  5.97074998 5.56428618
 6.38481692 4.70548583 5.19625119 5.02218227 6.46304341 6.02722075
 5.70059864 4.47669744 5.06849874 4.93354741 3.36710043 5.3456369
 6.5234563  3.39458816 5.65031866 6.08844343 5.57474319 6.00712079
 6.39995241 5.65274497 6.3904242

# Best Model

In [ ]:
import pickle

In [ ]:
def alltraits_models():

  all_traits = ['Overall','RecommendHiring','Colleague','Engaged','Excited','EyeContact','Smiled','SpeakingRate','NoFillers','Friendly','Paused','EngagingTone','StructuredAnswers','Calm','NotStressed','Focused','Authentic','NotAwkward']

  for trait in all_traits:
    model = training_SVR(trait=trait)
    pickle.dump(model , open('/content/drive/MyDrive/Grad Project/MIT/Models/' + str(trait) + '.sav' ,'wb'))

In [ ]:
alltraits_models()

(138, 40, 3)
(138, 120)
--------------------------------Overall--------------------------------
(0.5857795360207961, 0.001056088014881647)
[5.35879924 4.83062564 5.15237821 4.41713648 4.88619674 5.10823442
 4.3499188  5.07919638 4.56366144 5.11270411 5.46976898 4.35551465
 4.29326742 4.27634919 5.34596834 5.37545355 5.1421686  4.55747689
 5.33871661 5.26033267 4.20881739 5.19275724 4.84012629 4.62941405
 5.39727284 4.24728334 4.54260359 4.352233  ]
(138, 40, 3)
(138, 120)
--------------------------------RecommendHiring--------------------------------
(0.628035853096048, 0.00034601248592170905)
[5.1584754  4.63911752 5.20235331 4.40333802 4.78895839 4.9948733
 4.21146515 4.9281898  4.54616129 4.92724196 5.4191732  4.27137852
 4.17542645 4.14884148 5.08343278 5.1751775  5.17691828 4.52559344
 5.13981348 5.23246119 4.02035573 5.06188179 4.78726417 4.43092076
 5.46607197 4.2592242  4.37604697 4.19933962]
(138, 40, 3)
(138, 120)
--------------------------------Colleague---------------------

# Train with 15000 frames for 150 Secs (signal length = 7200000)

In [ ]:
model_svr = training_SVR('Overall')

(138, 14999, 40, 3)
(138, 1799880)
--------------------------------Overall--------------------------------
(0.04750395844070581, 0.8102975261802293)
[5.31344104 5.24981068 5.27450183 5.3913071  5.30837599 5.20054919
 5.23048399 5.10522398 5.23777569 5.08983979 5.1785183  4.95304589
 5.04869441 5.1373857  5.26220749 5.34873266 5.24882631 5.3274556
 5.53388865 5.09887235 5.20975654 5.30861839 5.28517681 5.07294331
 5.20108868 5.24325233 4.94276373 5.2157755 ]


In [ ]:
training_SVR('RecommendHiring')

(138, 14999, 40, 3)
(138, 1799880)
--------------------------------RecommendHiring--------------------------------
(0.12637923279699506, 0.5216368410998651)
[5.19227746 5.07676335 5.10716265 5.23682315 5.12977622 5.04706204
 5.06473642 4.92765334 5.05633705 4.96178347 5.03890735 4.83507819
 4.95029904 4.98772678 5.13402345 5.21806703 5.11022769 5.16479131
 5.41533825 4.97874027 5.03622585 5.1863223  5.14269247 4.92867628
 5.09310492 5.0272251  4.81288774 5.12867743]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# Train with less frames for 55 Secs (signal of length 3000000)

In [ ]:
training_SVR('Overall')

(138, 6249, 40, 3)
(138, 749880)
--------------------------------Overall--------------------------------
(0.097832250592118, 0.620410697700469)
[5.33454553 5.33695992 5.28677682 5.36165903 5.33612722 5.15048532
 5.21001906 5.02790599 5.18496029 5.01208388 5.14184122 4.9571353
 4.90917004 5.10676018 5.20908871 5.28803214 5.09999329 5.33236047
 5.49546465 5.04865013 5.24327454 5.26401383 5.31538117 4.99039675
 5.27437015 5.23750883 4.8595591  5.26353034]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 6249, 40, 3)
(138, 749880)
--------------------------------RecommendHiring--------------------------------
(0.22190564851764452, 0.25641733067882116)
[5.22577798 5.22506789 5.14632604 5.24727702 5.22999579 5.03488048
 5.07435    4.8538534  5.060077   4.87795037 5.01971453 4.81056457
 4.74384969 4.95507035 5.0614385  5.15808613 4.93523833 5.2194908
 5.35366908 4.85376212 5.10454135 5.14208013 5.20232754 4.84911173
 5.15912975 5.02176554 4.6666527  5.13647136]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# Train by taking average of all frames for a specific audio (40 * 3)

In [ ]:
training_SVR('Overall')

(138, 40, 3)
(138, 120)
--------------------------------Overall--------------------------------
(0.07435088575696806, 0.7069081113887687)
[5.1976063  5.19558021 5.09078756 5.38824526 5.05469682 5.50068974
 5.30937159 4.96054076 5.36706864 4.79348921 5.27340991 5.51137641
 4.31928306 5.34041538 5.18777376 5.44248891 5.3719069  5.72998326
 5.52620073 5.18575295 5.37855389 5.04626918 5.60558327 5.33800807
 5.4842907  5.38281178 4.48303201 5.33773423]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 40, 3)
(138, 120)
--------------------------------RecommendHiring--------------------------------
(0.33258655911522245, 0.0837619887346864)
[4.97840433 5.26846697 4.90015896 5.44192632 5.1803499  5.27283467
 5.24492303 4.84254672 5.40571913 4.73542199 5.1420447  5.24030937
 4.17749212 5.29505217 4.98475666 5.45031023 5.19574489 5.6469465
 5.37779978 5.12186167 5.30926944 5.05413575 5.66616434 5.13089343
 5.33653047 5.30335877 4.2885305  5.40931552]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## random state = 2

In [ ]:
training_SVR('Overall')

(138, 40, 3)
(138, 120)
--------------------------------Overall--------------------------------
(0.5857795360207961, 0.001056088014881647)
[5.35879924 4.83062564 5.15237821 4.41713648 4.88619674 5.10823442
 4.3499188  5.07919638 4.56366144 5.11270411 5.46976898 4.35551465
 4.29326742 4.27634919 5.34596834 5.37545355 5.1421686  4.55747689
 5.33871661 5.26033267 4.20881739 5.19275724 4.84012629 4.62941405
 5.39727284 4.24728334 4.54260359 4.352233  ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 40, 3)
(138, 120)
--------------------------------RecommendHiring--------------------------------
(0.628035853096048, 0.00034601248592170905)
[5.1584754  4.63911752 5.20235331 4.40333802 4.78895839 4.9948733
 4.21146515 4.9281898  4.54616129 4.92724196 5.4191732  4.27137852
 4.17542645 4.14884148 5.08343278 5.1751775  5.17691828 4.52559344
 5.13981348 5.23246119 4.02035573 5.06188179 4.78726417 4.43092076
 5.46607197 4.2592242  4.37604697 4.19933962]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## random state = 2 & Tuning

In [ ]:
training_SVR('Overall')

(138, 40, 3)
(138, 120)
--------------------------------Overall--------------------------------
(0.5857795360207961, 0.001056088014881647)
[5.35879924 4.83062564 5.15237821 4.41713648 4.88619674 5.10823442
 4.3499188  5.07919638 4.56366144 5.11270411 5.46976898 4.35551465
 4.29326742 4.27634919 5.34596834 5.37545355 5.1421686  4.55747689
 5.33871661 5.26033267 4.20881739 5.19275724 4.84012629 4.62941405
 5.39727284 4.24728334 4.54260359 4.352233  ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 40, 3)
(138, 120)
--------------------------------RecommendHiring--------------------------------
(0.628035853096048, 0.00034601248592170905)
[5.1584754  4.63911752 5.20235331 4.40333802 4.78895839 4.9948733
 4.21146515 4.9281898  4.54616129 4.92724196 5.4191732  4.27137852
 4.17542645 4.14884148 5.08343278 5.1751775  5.17691828 4.52559344
 5.13981348 5.23246119 4.02035573 5.06188179 4.78726417 4.43092076
 5.46607197 4.2592242  4.37604697 4.19933962]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 40, 3)
(138, 120)
--------------------------------Friendly--------------------------------
(0.7682971435266075, 1.8066191038799882e-06)
[5.39002441 5.04287232 5.88955716 4.93512232 5.17863499 5.31252755
 4.4331773  5.20722761 4.72337413 5.38231383 5.6557756  4.47298861
 4.38558839 4.65560662 5.4405703  5.38001925 5.8689138  5.07248399
 5.25047753 5.71380484 4.30527324 5.20876061 5.03134362 5.00203585
 5.74744201 4.63201792 4.77367071 4.44821202]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 40, 3)
(138, 120)
--------------------------------Engaged--------------------------------
(0.5564023113806897, 0.0021065984409310854)
[5.3933965  4.9612051  5.8563023  4.79775128 5.2914514  5.50556965
 4.77756294 5.45581693 4.89872918 5.69536275 5.8133311  4.70747664
 4.75056681 4.72729793 5.58302697 5.5899456  5.82626333 4.95727819
 5.43630924 5.81963679 4.74641315 5.6387903  5.07342767 5.18597904
 5.95031856 4.65568319 4.97846058 4.6615305 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 40, 3)
(138, 120)
--------------------------------Excited--------------------------------
(0.7605081362257875, 2.6457802431389195e-06)
[4.72258247 4.23740313 5.38350631 3.86894064 4.8056114  4.89549233
 3.80737379 4.64252389 4.14161366 4.97668376 5.26554881 3.74838221
 3.82437303 3.57641391 5.07526357 4.86210207 5.47572217 4.02249727
 4.7541703  5.28945882 3.83500364 4.90026904 4.28092891 4.29132244
 5.35886841 3.54822298 4.0207818  3.80620221]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 40, 3)
(138, 120)
--------------------------------EngagingTone--------------------------------
(0.8057332276382179, 2.300001933216751e-07)
[4.77042043 4.03845249 5.34347688 3.55755344 4.63523099 5.14167695
 3.54503289 4.58414208 3.86041701 5.10998456 5.12379966 3.51197232
 3.49561426 3.32519414 5.11038193 5.12604977 5.27707394 3.83304716
 4.93008064 5.46304316 3.54683168 4.84193023 4.13955613 4.12041384
 5.65591074 3.1864576  3.89099726 3.43994316]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## random state = 42 & Tuning

In [ ]:
training_SVR('Overall')

(138, 40, 3)
(138, 120)
--------------------------------Overall--------------------------------
(0.18792729895223823, 0.3382396035882068)
[5.57352576 4.75723273 5.27160178 5.26827314 5.04321379 5.54399918
 4.95813801 5.26022222 5.37233827 5.68014463 5.5918675  5.50646949
 4.83475168 5.13013901 5.20003882 5.01353965 5.28604826 5.79718757
 4.97992871 5.41923914 4.94490174 5.28715053 5.38458369 5.52001167
 5.44645662 5.38673154 5.10704251 5.06215947]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 40, 3)
(138, 120)
--------------------------------RecommendHiring--------------------------------
(0.3031540917607299, 0.11684227445541621)
[5.25659804 4.32639555 4.94229609 5.19131873 4.70339148 5.53734129
 4.38575169 5.02820672 5.03056318 5.53481724 5.5368832  5.57797817
 4.80018951 4.75476249 4.86879576 4.72266975 5.08351891 5.82830708
 4.95558236 5.35315696 4.65376029 5.10751083 5.15973602 5.41631843
 5.1529412  5.35666391 4.79827209 4.78240453]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 40, 3)
(138, 120)
--------------------------------Friendly--------------------------------
(0.4714686934486432, 0.011320101730759627)
[5.44100706 5.02014019 5.52969618 5.57399035 5.53545922 5.656351
 4.88802206 5.44136471 5.63709159 6.08759939 5.54432002 5.49707444
 5.1748759  5.45744869 5.46575581 5.10466864 5.61119231 5.84330695
 5.63777472 5.5246444  5.19315137 5.49199487 5.61107029 6.0033264
 5.53987574 5.47480387 5.66784859 5.29308216]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 40, 3)
(138, 120)
--------------------------------Engaged--------------------------------
(0.6007959469734941, 0.0007230850909625136)
[5.72801154 4.93164938 5.7992958  5.73433059 5.61803917 5.96539023
 5.08512395 5.33588271 5.45564382 5.79340959 5.44408255 5.462203
 5.02809698 5.86062288 5.44778669 5.01978633 5.85476622 6.0257181
 5.73925375 5.84352749 5.29884941 5.49703924 5.46966287 5.85616727
 5.61856666 5.53127891 5.43911812 5.61408165]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 40, 3)
(138, 120)
--------------------------------Excited--------------------------------
(0.7510752795264453, 4.122309814708578e-06)
[5.17487527 3.83669258 4.95243134 5.19306936 4.61604108 5.60866352
 4.0494152  4.71567333 5.10161502 5.2850091  4.65457543 5.18478383
 4.41357345 5.31244179 4.78628027 4.32149234 5.39570919 6.04755744
 5.18854157 5.32396887 4.34174753 4.49233213 4.79278849 5.22876953
 4.93117632 4.98761394 4.64089869 4.91583969]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 40, 3)
(138, 120)
--------------------------------EngagingTone--------------------------------
(0.7215109328112013, 1.472439549125262e-05)
[5.19279937 3.95510368 5.30148153 5.02701501 5.10672981 5.45295735
 4.26140116 4.77901189 4.89293291 5.39044768 4.84886041 4.67774387
 4.16837721 5.33816668 4.75429827 4.09120963 5.46068463 5.73629407
 5.20841135 5.23010488 4.58930224 4.96635474 4.80332785 5.39499241
 5.16111407 4.8367829  4.92385976 4.91821484]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)